In [1]:
import numpy as np
import pandas as pd

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from khaiii import KhaiiiApi

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

import re
import pickle
import tool
from pprint import pprint

In [2]:
song = pd.read_json("res/song_meta.json")[["id", "song_name", "artist_id_basket", "artist_name_basket",\
                                            "album_id", "album_name", "song_gn_gnr_basket",\
                                            "song_gn_dtl_gnr_basket", "issue_date"]]
genre = pd.read_json("res/genre_gn_all.json", typ="series")
train = pd.read_json("res/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val = pd.read_json("res/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
test = pd.read_json("res/test.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
song.name = "song"
genre.name = "genre"
train.name = "train"
val.name = "val"
test.name = "test"
display(song, genre, train, val, test)

,id,song_name,artist_id_basket,artist_name_basket,album_id,album_name,song_gn_gnr_basket,song_gn_dtl_gnr_basket,issue_date
0,0,Feelings,[2727],[Various Artists],2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,[GN0900],[GN0901],20140512
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],[Murray Perahia],376431,"Bach : Partitas Nos. 2, 3 & 4",[GN1600],"[GN1601, GN1606]",20080421
2,2,Solsbury Hill (Remastered 2002),[3361],[Peter Gabriel],4698747,Hit,[GN0900],[GN0901],20180518
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[838543],[Matoma],2644882,Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],"[GN1102, GN1101]",20151016
4,4,그남자 그여자,[560160],[Jude Law],2008470,그남자 그여자,[GN1800],"[GN1802, GN1801]",20110824
...,...,...,...,...,...,...,...,...,...
707984,707984,Coffin For Head Of State,[166499],[Fela Kuti],65254,The Best Best Of The Black President,[GN2000],[GN2001],19991219
707985,707985,Change Of Heart,[11837],[Cyndi Lauper],44141,True Colors,[GN0900],[GN0901],19860000
707986,707986,스치듯 안녕,[437],[윤종신],2662866,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,[GN0100],"[GN0105, GN0101]",20160120
707987,707987,숲의 빛,[729868],[Nature Piano],2221722,명상의 시간을 위한 뉴에이지 음악,[GN1800],"[GN1807, GN1801]",20131217


GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Name: genre, Length: 254, dtype: object

,id,plylst_title,songs,tags,like_cnt,updt_date
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",17,2017-08-28 07:09:34.000
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",33,2019-12-05 15:15:18.000
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",3,2020-04-17 04:31:11.000
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",[일렉],13,2015-12-24 17:23:19.000
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",4,2019-08-16 20:59:22.000
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",4,2019-10-25 23:40:42.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],1675,2019-05-27 14:14:33.000
1,131447,앨리스테이블,[],[],1,2014-07-16 15:24:24.000
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],62,2008-06-21 23:26:22.000
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],20,2017-10-30 18:15:43.000
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[잔잔한],17,2015-12-17 14:06:05.000
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",10,2020-04-16 21:35:44.000
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[],0,2019-03-27 15:27:40.000
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[],31,2015-11-18 11:49:09.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[],6,2012-09-29 01:57:26.000
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[나만의Best3, 인디아티스트들의추천음악]",0,2019-12-17 14:06:45.000
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브],21,2015-05-23 10:44:48.000
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[분위기],0,2019-03-14 09:47:34.000
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[],38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[추억],16,2016-04-18 11:02:09.000
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[띵곡의, 우울, 분위기, 드라이브, 산책]",1,2020-04-08 07:15:59.000
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",[기분전환],6,2016-06-29 00:57:21.000
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[여름],4,2015-06-06 09:52:01.000


### Separate Korean and English from *playlist-title* and *tags*

In [10]:
%%time
run = False
data = train
dtype = "str"
if run:
    ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dense=True, limit=None)
    with open(f'bin/{dtype}_{data.name}.p', 'wb') as f:
        _tuple = (ko_title, ko_tag, en_title, en_tag)
        pickle.dump(_tuple, f)
if not run: # data load
    with open(f'bin/{dtype}_{data.name}.p', 'rb') as f:
        ko_title, ko_tag, en_title, en_tag = pickle.load(f)
ko_title, ko_tag, en_title, en_tag

CPU times: user 102 ms, sys: 27.9 ms, total: 129 ms
Wall time: 129 ms


({0: '여행같은 음악',
  1: '요즘 너 말야',
  2: '편하게 잔잔하게 들을 수 있는 곡',
  3: '크리스마스 분위기에 흠뻑 취하고 싶을때',
  4: '추억의 노래 ㅋ',
  6: '짝사랑고백사랑이별슬픔 감성을 자극하는곡들',
  7: '멍청이 내맘도 몰라',
  9: '록메탈   이일우',
  10: '걸그룹 땐쓰쏭',
  11: '노래로 의지를 불태우자 일다짐 용 프로필뮤직',
  12: '지친 너를 위로해줄 제목편지 준비해봤어',
  13: '트렌디하고 그루브한 힙합알앤비',
  15: '걍게임할때듣는음악',
  16: '나만 알고싶은 노래들',
  17: '축가듀엣',
  18: '이렇게 비 내리는 날이면 너도 내 생각 할까',
  19: '올림픽 스밍 목록',
  20: '조용히 맥주 한잔 할때새벽감성',
  22: '추억의 명화를 만나는 시간 영화  모음',
  23: '기분좋은 여름날',
  24: '지친 하루 끝 힐링이 필요한 당신에게 추천하는 인디곡',
  25: '새벽 찬바람이 침대위를 감쌀때국외',
  26: '기분좋은 햇살같은 숨겨진 명곡들',
  27: '사랑스러운 남녀듀엣',
  28: '즐겨듣는소래',
  29: '닥힙추 월',
  30: '거슈윈에서 스타워즈까지 미국 대표 클래식',
  31: '겨울이 기다리며 미리 들어요',
  32: '아침이 상쾌해지는 빅밴드의 스윙스윙',
  33: '여행가는 길 이런음악과 함께 고고씽',
  34: '믿고듣는 하우스 일렉트로니카',
  35: '프로포즈 음악',
  36: '보아일본곡',
  37: '안다르 레깅스 입고 운동 시작하자',
  38: '발라드로 감성을 발라드림슬픔주의',
  39: '크리스마스를 책임질 캐롤 모음',
  40: '아무것도 위로 되지 못할 때 위로가 되어준 음악',
  41: '트렌디한',
  42: '년대 우리나라 라디오에서 흘러나오던 명곡들',
  43: '영화 를 만나다',
  44: '우울하고 시크하',
  45: '극도로 우울할때',
 

### Separate Korean and English from *song-name*, *name-of-artists* and *album-name*

In [3]:
%%time
run = False
if run:
    dtype = "str"
    ko_name, ko_album, ko_artist, en_name, en_album, en_artist = tool.extract_song_from(song, dtype=dtype, limit=None)
    with open(f'bin/{dtype}_song.p', 'wb') as f:
        _tuple = (ko_name, ko_album, ko_artist, en_name, en_album, en_artist)
        pickle.dump(_tuple, f)
if not run: # data load
    with open(f'bin/{dtype}_song.p', 'rb') as f:
        ko_name, ko_album, ko_artist, en_name, en_album, en_artist = pickle.load(f)
ko_name, ko_album, ko_artist, en_name, en_album, en_artist


CPU times: user 47.4 s, sys: 894 ms, total: 48.3 s
Wall time: 48.1 s


({4: '그남자 그여자',
  6: '시벨리우스  슬픈 왈츠 작품번호',
  9: '사랑 그대라는 멜로디',
  18: '숙면과 휴식에 좋은 편안한 빗소리 백색소음',
  19: '무얼 기다리나  조원선',
  20: '길병민 유채훈 한기주 팬텀싱어  유채훈 가창곡',
  25: '뒤돌아봐',
  29: '창밖에 잠수교가 보인다',
  32: '먼지',
  35: '찾고 있니',
  36: '연가',
  48: '나의 노래',
  49: '야상곡',
  51: '초콜렛 먹는 소리 고요함',
  53: '내가 그리스도와 함께',
  54: '있어줄래',
  58: '국화꽃 향기  김세찬',
  59: '고향의 봄',
  60: '논현   최재문',
  62: '잠시 길을 잃다  신보경',
  65: '차이콥스키  잠자는 숲속의 미녀 작품번호   번 파노라마',
  66: '그리움으로 물든 노을',
  67: '떠나지 못해',
  68: '나를 위한 휴식',
  70: '너에게 전하지 못한 말',
  81: '두 번째 인생',
  85: '매미의 꿈      당신을 위로하고 싶은 마음에서 나온 가벼운 성의',
  86: '우리 사랑 이야기',
  92: '참쓰다',
  96: '가르마',
  97: '따뜻한 바람 부는 계절',
  99: '장일남  비목',
  105: '하늘은 말없이',
  111: '교생쌤',
  112: '예수를 나의 구주삼고',
  115: '마음의 소리',
  124: '드뷔시  베르가마스크 모음곡  번 달빛 명상음악 클래식',
  127: '걱정말아요 그대',
  130: '하이든  현악 중주 번 바장조 작품번호   악장',
  131: '보이비',
  133: '충무로의 밤',
  135: '그런 사람',
  136: '나 주를 멀리 떠났다',
  143: '커피 한잔 할래요',
  146: '러빙러빙',
  148: '어디쯤 있나요',
  149: '겨울 애상',
  152: '라 메르   엔딩 타이틀',
  157: '사랑의 이름

### Tokenize

In [ ]:
api = KhaiiiApi()

In [4]:
api = KhaiiiApi()
morphses = []
def morphs(api, sentence):
    m = []
    print(sentence)
    for word in api.analyze(sentence):
        print(word)
        for morph in word.morphs:
            m.append((morph.lex, morph.tag))
    return m

# for i in train.index:
#     sentence = train.plylst_title[i]
#     morphs = []
#     print(f"{i}th playlist")
#     for word in api.analyze(sentence):
#         print(word)
#         for morph in word.morphs:
#             morphs.append((morph.lex, morph.tag))
#     morphses.append(morphs)
#     if i == 3:
#         break
morphses
morphs(api, "hello world세상 안녕 세상")

hello world세상 안녕 세상
hello	hello/SL
world세상	world/SL + 세상/NNG
안녕	안녕/MAG
세상	세상/NNG


[('hello', 'SL'), ('world', 'SL'), ('세상', 'NNG'), ('안녕', 'MAG'), ('세상', 'NNG')]

In [25]:
s='韓子는 싫고, 한글은 nice하다. English 쵝오 -_-ㅋㅑㅋㅑ ./?!'
s = 'allenglish'
p = re.compile('[ㄱ-ㅣ가-힣]+')
print(p.findall(s))
q = re.compile('[^ ㄱ-ㅣ가-힣]+')
t = q.sub('', s)
t == ''

[]


True